# Rating Product & Sorting Reviews in Amazon

## Business Problem

One of the most important problems in e-commerce is the correct calculation of the points given to the products after sales. The solution to this problem means providing greater customer satisfaction for the e-commerce site, prominence of the product for the sellers and a seamless shopping experience for the buyers. Another problem is the correct ordering of the comments given to the products. Since misleading comments will directly affect the sale of the product, it will cause both financial loss and loss of customers. In the solution of these 2 basic problems, while the e-commerce site and the sellers will increase their sales, the customers will complete the purchasing journey without any problems.

### Dataset Story


This dataset, which includes Amazon product data, includes product categories and various metadata. The product with the most reviews in the electronics category has user ratings and reviews.

**reviewerID:** User ID

**asin:** Product ID

**reviewerName:** Username

**Helpful:** Useful evaluation rating

**reviewText:** Evaluation

**overall:** Product rating

**summary:** Evaluation summary

**unixReviewTime:** Evaluation time

**reviewTime:** Review time Raw

**day_diff:** Number of days since evaluation

**helpful_yes:** Number of times the review was found helpful

**total_vote:** Number of votes given to the review

In [1]:
#Necessary libraries are imported.
import pandas as pd
import math
import scipy.stats as st
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_csv("../input/amazon-review/amazon_review.csv")


In [3]:
df = df_.copy()


# Data Overview

In [4]:
#The first 5 rows of the dataset were looked at.
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [5]:
#The last 5 rows of the dataset were viewed.
df.tail()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
4910,A2LBMKXRM5H2W9,B007WTAJTO,"ZM ""J""","[0, 0]",I bought this Sandisk 16GB Class 10 to use wit...,1.00000,Do not waste your money.,1374537600,2013-07-23,503,0,0
4911,ALGDLRUI1ZPCS,B007WTAJTO,Zo,"[0, 0]",Used this for extending the capabilities of my...,5.00000,Great item!,1377129600,2013-08-22,473,0,0
4912,A2MR1NI0ENW2AD,B007WTAJTO,Z S Liske,"[0, 0]",Great card that is very fast and reliable. It ...,5.00000,Fast and reliable memory card,1396224000,2014-03-31,252,0,0
4913,A37E6P3DSO9QJD,B007WTAJTO,Z Taylor,"[0, 0]",Good amount of space for the stuff I want to d...,5.00000,Great little card,1379289600,2013-09-16,448,0,0
4914,A8KGFTFQ86IBR,B007WTAJTO,Zza,"[0, 0]",I've heard bad things about this 64gb Micro SD...,5.00000,So far so good.,1388620800,2014-02-01,310,0,0


In [6]:
#Variable types and numbers of the data set were examined.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [7]:
#It was checked for missing data in the dataset.
df.isnull().sum()

reviewerID        0
asin              0
reviewerName      1
helpful           0
reviewText        1
overall           0
summary           0
unixReviewTime    0
reviewTime        0
day_diff          0
helpful_yes       0
total_vote        0
dtype: int64

In [8]:
#The statistical properties of the columns of the data set containing numeric data were checked.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.00000,4.58759,0.99685,1.00000,5.00000,5.00000,5.00000,5.00000
unixReviewTime,4915.00000,1379465001.66836,15818574.32275,1339200000.00000,1365897600.00000,1381276800.00000,1392163200.00000,1406073600.00000
day_diff,4915.00000,437.36704,209.43987,1.00000,281.00000,431.00000,601.00000,1064.00000
helpful_yes,4915.00000,1.31109,41.61916,0.00000,0.00000,0.00000,0.00000,1952.00000
total_vote,4915.00000,1.52146,44.12309,0.00000,0.00000,0.00000,0.00000,2020.00000


In [9]:
#The number of unique values for each variable in the dataset was looked at.
df.nunique()

reviewerID        4915
asin                 1
reviewerName      4594
helpful             42
reviewText        4912
overall              5
summary           3885
unixReviewTime     690
reviewTime         690
day_diff           690
helpful_yes         23
total_vote          26
dtype: int64

# Product Rating

In [10]:
#The average score of the product was calculated.
df.overall.mean()

4.587589013224822

In [11]:
#The data types of the date variables are changed with datetime module to be able to use them in date calculations.
df["reviewTime"] = df["reviewTime"].apply(pd.to_datetime)

current_date = df["reviewTime"].max()

df["day_dif"] = (current_date - df["reviewTime"]).astype('timedelta64[D]')



In [12]:
#Quarterly values were found to determine weight.
df["day_dif"].quantile([.25, .5, .75])

0.25000   280.00000
0.50000   430.00000
0.75000   600.00000
Name: day_dif, dtype: float64

In [13]:
#Calculated the weighted mean score by date.
def time_based_weighted_average(dataframe, w1=30, w2=28, w3=24, w4=18):
    return dataframe.loc[dataframe["day_dif"] <= 280, "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_dif"] > 280) & (dataframe["day_dif"] <=430), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_dif"] > 430) & (dataframe["day_dif"] <=600), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_dif"] > 600), "overall"].mean() * w4 / 100

time_based_weighted_average(df)

4.604381672622366

In [14]:
#Average of ratings made before 280 days.
df.loc[df["day_dif"] <= 280, "overall"].mean()

4.6957928802588995

In [15]:
#Average of ratings made between 280 days and 430 days.
df.loc[(df["day_dif"] > 280) & (df["day_dif"] <=430), "overall"].mean()

4.636140637775961

In [16]:
#Average of ratings made between 430 days and 600 days.
df.loc[(df["day_dif"] > 430) & (df["day_dif"] <= 600), "overall"].mean()

4.571661237785016

In [17]:
#Average of ratings over 600 days.
df.loc[(df["day_dif"] > 600), "overall"].mean()

4.4462540716612375

In [18]:
#how many people were found in each scoring range.
df.groupby("asin").agg({"overall" : "value_counts"})

overall
asin       overall         
B007WTAJTO 5.00000     3922
           4.00000      527
           1.00000      244
           3.00000      142
           2.00000       80

In [19]:
#The Bayesian average adjusts the average rating of products whose rating counts fall below a threshold. 
def bayesian_average_rating(n, confidence=0.95):
    if sum(n) == 0:
        return 0
    K = len(n)
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    N = sum(n)
    first_part = 0.0
    second_part = 0.0
    for k, n_k in enumerate(n):
        first_part += (k + 1) * (n[k] + 1) / (N + K)
        second_part += (k + 1) * (k + 1) * (n[k] + 1) / (N + K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score

In [20]:
bayesian_average_rating([244,80,142,526,3921])

4.558101840050464

# Product Reviews

In [21]:
#The number of those who did not find the comment useful was calculated.
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [22]:
df["helpful_no"].value_counts()

0      4674
1       175
2        43
3         7
27        2
4         2
6         2
73        1
8         1
10        1
68        1
110       1
183       1
77        1
126       1
14        1
9         1
Name: helpful_no, dtype: int64

In [23]:
#Scoring according to the difference between up and down
def score_up_down_diff(up, down):
    return up - down

df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"],x["helpful_no"]), axis=1 )

In [24]:
df.sort_values(by = "score_pos_neg_diff", ascending = False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_dif,helpful_no,score_pos_neg_diff
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701.00000,68,1884
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578.00000,126,1442
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802.00000,77,1351
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032.00000,73,349
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.00000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,776.00000,27,85


In [25]:
#It scores the comment by dividing the number of ups by the total number of votes.
def score_average_rating(up, down):
  if up + down == 0:
    
      return 0

  return up / (up+down)

df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"],x["helpful_no"]), axis=1 )

In [26]:
df.sort_values(by = "score_average_rating", ascending = False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_dif,helpful_no,score_pos_neg_diff,score_average_rating
4277,A35KXSU6AD1481,B007WTAJTO,S. Q.,"[1, 1]",I have a galaxy note II and after rooting I no...,5.00000,Perfect!!,1355875200,2012-12-19,719,1,1,718.00000,0,1,1.00000
2881,A3VSG5X7GPNNW6,B007WTAJTO,Lou Thomas,"[1, 1]",The Nexus One is listed as supporting a maximu...,5.00000,Nexus One Loves This Card!,1349049600,2012-01-10,1063,1,1,1062.00000,0,1,1.00000
1073,A2ZXEKQ2OBZLEE,B007WTAJTO,C. Sanchez,"[1, 1]",I used it with my Samsung S4 and it works grea...,5.00000,Tons of space for phone,1376352000,2013-08-13,482,1,1,481.00000,0,1,1.00000
445,AIWBDRNBODLEA,B007WTAJTO,"Apache ""Elizabeth""","[1, 1]",This is exactly what I was looking for to upgr...,4.00000,Amazon Great Prices,1387324800,2013-12-18,355,1,1,354.00000,0,1,1.00000
3923,A2PH4RGYVR34L,B007WTAJTO,Rock Your Roots,"[1, 1]","It's a SanDisk, so what more is there to say? ...",5.00000,What more to say?,1388361600,2013-12-30,343,1,1,342.00000,0,1,1.00000


In [27]:
"""
     Calculate Wilson Lower Bound Score

     - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
     - The score to be calculated is used for product ranking.
     - Note:
     If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and can be made to conform to Bernoulli.
     This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.

     parameters
     ----------
     up: int
         up count
     down: int
         down count
     confidence: float
         confidence

     Returns
     -------
     wilson score: float

     """
def wilson_lower_bound(up, down, confidence=0.95):

    n = up + down
    if n == 0:
        return 0 

    z = st.norm.ppf(1 - (1 - confidence) / 2)
    
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"],x["helpful_no"]), axis=1 )

In [28]:
df.sort_values(by = "wilson_lower_bound", ascending = False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,day_dif,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701.00000,68,1884,0.96634,0.95754
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802.00000,77,1351,0.94884,0.93652
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578.00000,126,1442,0.92562,0.91214
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.00000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032.00000,73,349,0.85253,0.81858
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157.00000,4,41,0.91837,0.80811
